In [6]:
import cv2
import datetime
from collections import deque
from fer import FER
import pygame
import os
import csv
import face_recognition
import numpy as np

# Initialize Pygame for playing audio
pygame.mixer.init()

# Load the cascades for face, eyes, and smiles
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')
smile_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_smile.xml')

# Check if cascades are loaded properly
if face_cascade.empty():
    print("Error loading face cascade")
if eye_cascade.empty():
    print("Error loading eye cascade")
if smile_cascade.empty():
    print("Error loading smile cascade")

# Initialize the video capture object
video_capture = cv2.VideoCapture(0)

# Define the minimum face size for detection
min_face_size = (30, 30)

# Initialize FER detector
emotion_detector = FER(mtcnn=True)

# Initialize deque to store detected emotions
emotion_history = deque(maxlen=10)

# Initialize CSV file for saving face data
csv_filename = 'stored_faces/face_data.csv'
fieldnames = ['image_path', 'name', 'emotion', 'attention_state', 'timestamp']

# Create the directory and CSV file if they don't exist
os.makedirs('stored_faces', exist_ok=True)
if not os.path.isfile(csv_filename):
    with open(csv_filename, 'w', newline='') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()

# Load images and create face encodings for each image in the Image folder
image_folder = 'Image'
known_face_encodings = []
known_face_names = []

for filename in os.listdir(image_folder):
    if filename.endswith(".jpg") or filename.endswith(".png"):
        image_path = os.path.join(image_folder, filename)
        image = face_recognition.load_image_file(image_path)
        face_encoding = face_recognition.face_encodings(image)[0]  # Assuming one face per image
        known_face_encodings.append(face_encoding)
        known_face_names.append(os.path.splitext(filename)[0])

# Function to play siren sound
def play_siren():
    try:
        pygame.mixer.music.load('siren.mp3')
        pygame.mixer.music.play()
    except pygame.error as e:
        print(f"Error playing siren: {e}")

# Function to save detected faces and associated data
def save_faces(frame, faces, recognized_names, current_emotions, attention_states):
    data_to_write = []
    for i, (x, y, w, h) in enumerate(faces):
        face = frame[y:y+h, x:x+w]
        target_dir = 'stored_faces'
        os.makedirs(target_dir, exist_ok=True)  # Ensure directory exists
        image_path = os.path.join(target_dir, f'face_{datetime.datetime.now().strftime("%Y%m%d_%H%M%S")}_{i}.jpg')
        cv2.imwrite(image_path, face)
        print(f"Saved face {i} to {image_path}")

        # Save data to CSV
        data_to_write.append({
            'image_path': image_path,
            'name': recognized_names[i] if recognized_names else 'Unknown',
            'emotion': current_emotions[i] if current_emotions else 'Unknown',
            'attention_state': attention_states[i] if attention_states else 'Unknown',
            'timestamp': datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        })
    
    # Write data to CSV file
    with open(csv_filename, 'a', newline='') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        for data in data_to_write:
            writer.writerow(data)

# Determine attention state based on emotion
def determine_attention_state(emotion, eye_detected, smile_detected):
    if eye_detected:
        if smile_detected:
            return "Happy"
        elif emotion in ['neutral', 'happy']:
            return "Attentive"
        else:
            return "Neutral"
    else:
        if emotion in ['neutral', 'sad', 'angry']:
            return "Lazy"
        else:
            return "Distracted"

# Flag to keep track of siren state
is_playing_siren = False

while True:
    # Capture frame-by-frame
    ret, frame = video_capture.read()
    if not ret:
        print("Failed to grab frame")
        break

    # Resize the frame to speed up processing
    frame = cv2.resize(frame, (640, 480))

    # Convert the frame to grayscale for faster processing
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Apply histogram equalization to improve contrast
    gray = cv2.equalizeHist(gray)

    # Detect faces in the grayscale frame
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5, minSize=min_face_size)

    # Draw a rectangle around each face detected, and detect eyes and smiles within each face
    recognized_names = []
    current_emotions = []
    attention_states = []

    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = frame[y:y+h, x:x+w]

        # Detect eyes in the face region
        eyes = eye_cascade.detectMultiScale(roi_gray, scaleFactor=1.1, minNeighbors=5)
        eye_detected = len(eyes) > 0

        # Draw rectangles around the detected eyes
        for (ex, ey, ew, eh) in eyes:
            cv2.rectangle(roi_color, (ex, ey), (ex+ew, ey+eh), (0, 0, 255), 2)

        # Initialize smile_detected as False
        smile_detected = False

        # Detect smiles in the face region
        smiles = smile_cascade.detectMultiScale(roi_gray, scaleFactor=1.5, minNeighbors=15, minSize=(25, 25))

        # Check for teeth showing in the smile area for more accurate smile detection
        for (sx, sy, sw, sh) in smiles:
            smile_roi = gray[y+sy:y+sy+sh, x+sx:x+sx+sw]
            _, smile_thresh = cv2.threshold(smile_roi, 25, 255, cv2.THRESH_BINARY)
            teeth_pixels = cv2.countNonZero(smile_thresh)
            if teeth_pixels > 10:  # Adjust threshold as needed
                cv2.rectangle(roi_color, (sx, sy), (sx+sw, sy+sh), (255, 0, 0), 2)
                smile_detected = True
                break

        # Emotion detection using FER
        emotion, _ = emotion_detector.top_emotion(frame[y:y+h, x:x+w])
        if smile_detected:
            current_emotion = 'happy'
        elif emotion:
            emotion_history.append(emotion)
            current_emotion = max(set(emotion_history), key=emotion_history.count)
        else:
            current_emotion = None

        # Determine attention state
        attention_state = determine_attention_state(current_emotion, eye_detected, smile_detected)

        # Face recognition
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB
        face_locations = [(y, x+w, y+h, x)]  # Convert (x, y, w, h) to (top, right, bottom, left)
        face_encodings = face_recognition.face_encodings(rgb_frame, face_locations)

        for face_encoding in face_encodings:
            matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
            name = "Unknown"
            if True in matches:
                first_match_index = matches.index(True)
                name = known_face_names[first_match_index]

            recognized_names.append(name)
            current_emotions.append(current_emotion)
            attention_states.append(attention_state)

            # Display name on the frame
            cv2.putText(frame, f"Name: {name}", (x, y - 70), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

        # Display emotion and attention state on the frame
        if current_emotion:
            cv2.putText(frame, f"Emotion: {current_emotion}", (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 255, 255), 2)
        cv2.putText(frame, f"Attention: {attention_state}", (x, y+h+30), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 255, 255), 2)

    # Save detected faces and data
    save_faces(frame, faces, recognized_names, current_emotions, attention_states)

    # Play siren sound if face count exceeds 1
    if len(faces) > 1:
        if not is_playing_siren:
            play_siren()
            is_playing_siren = True
    else:
        if is_playing_siren:
            pygame.mixer.music.stop()
            is_playing_siren = False

    # Display the count and datetime on the frame
    datetime_now = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    cv2.putText(frame, f"Face Count: {len(faces)}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.putText(frame, f"Date and Time: {datetime_now}", (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    # Display the resulting frame
    cv2.imshow('Video', frame)

    # Press 'q' to exit the loop and stop the video capture
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture object and close all windows
video_capture.release()
cv2.destroyAllWindows()



Saved face 0 to stored_faces/face_20240628_032722_0.jpg
Saved face 0 to stored_faces/face_20240628_032723_0.jpg
Saved face 0 to stored_faces/face_20240628_032723_0.jpg
Saved face 0 to stored_faces/face_20240628_032723_0.jpg
Saved face 0 to stored_faces/face_20240628_032723_0.jpg
Saved face 0 to stored_faces/face_20240628_032724_0.jpg
Saved face 0 to stored_faces/face_20240628_032724_0.jpg
Saved face 0 to stored_faces/face_20240628_032724_0.jpg
Saved face 0 to stored_faces/face_20240628_032726_0.jpg
Saved face 0 to stored_faces/face_20240628_032726_0.jpg
Saved face 0 to stored_faces/face_20240628_032727_0.jpg
Saved face 0 to stored_faces/face_20240628_032727_0.jpg
Saved face 0 to stored_faces/face_20240628_032727_0.jpg
Saved face 0 to stored_faces/face_20240628_032728_0.jpg
Saved face 0 to stored_faces/face_20240628_032728_0.jpg
Saved face 0 to stored_faces/face_20240628_032728_0.jpg
Saved face 0 to stored_faces/face_20240628_032728_0.jpg
Saved face 0 to stored_faces/face_20240628_03272

KeyboardInterrupt: 

: 